# Decorators
Yang Xi <br>
14 Sep, 2020


<br>


* Introduction
* Knowledge Refresher
	* Important Aspects of Python Functions
* Function Decorators
	* A Simple Function Decorator
	* Use Syntax @
	* Decorate 3rd Party Functions
	* Generalized Function Wrapper
	* Decorators with Parameters
	* Preserve Attributes of Decorated Function
* Class Decorators
* Use Cases
	* Checking Arguments
	* Counting Function Calls
	* Memoization with Decorators

# Introduction
A **decorator** is any callable Python object that is used to **modify a function, method or a class**. A reference ot a function or a class is passed to a decorator and the decorator returns a modified function or class.

Two kinds of decorators:
* Function decorators
* Class decorators

Reference: https://www.python-course.eu/python3_decorators.php

# Knowledge Refresher
### Important Aspects of Python Functions
* Function names are references to functions - we can assign multiple names to the same function.
* We can delete a function name without deleting the function itself.
* Functions can be defined inside a function.
* We can pass functions as parametmers to a function.
    * Can use the attribute __name__ to identify the 'real' name




In [1]:
def g():
    print('this is g')

def f(func):
    print('this is f')
    func()
    print("func's real name is "+func.__name__)

f(g)

this is f
this is g
func's real name is g


* Functions can return references to function objects.

In [2]:
def f(x):
    def g(y):
        return y + x + 3 
    return g

nf1 = f(1)
nf2 = f(3)

print(nf1(1))
print(nf2(1))

5
7


# Function Decorators
### A Simple Function Decorator

In [3]:
def our_decorator(func):
    def function_wrapper(x):
        print("Do something additional...")
        func(x)
    return function_wrapper

def foo(x):
    print("Call foo with " + str(x))

print("Call foo before decoration:")
foo("Hi")

print("\nDecorate foo with f...")
foo = our_decorator(foo)

print("\nCall foo after decoration:")
foo("Hi")

Call foo before decoration:
Call foo with Hi

Decorate foo with f...

Call foo after decoration:
Do something additional...
Call foo with Hi


### Use Syntax @
The decoration occurs in the line before the function header.
* The **@** is followed by decortor function name
* This line has to be directly positioned in front of the decorated function

In [4]:
def our_decorator(func):
    def function_wrapper(x):
        print("Do something additional...")
        func(x)
    return function_wrapper

@our_decorator
def foo(x):
    print("Call foo with " + str(x))

foo("Hi")

Do something additional...
Call foo with Hi


### Decorate 3rd Party Functions
It is also possible to decorate 3rd party functions, e.g. functions we import from a module. But we **cannot use @** in this case.

In [5]:
from math import cos

def our_decorator(func):
    def function_wrapper(x):
        print("Do something additional...")
        res = func(x)
        print(res)
    return function_wrapper

cos = our_decorator(cos)

cos(3.1415)

Do something additional...
-0.9999999957076562


### Generalized Function Wrapper

In [6]:
from math import cos

def our_decorator(func):
    def function_wrapper(*args, **kwargs):
        print("Do something additional...")
        res = func(*args, **kwargs)
        print(res)
    return function_wrapper

cos = our_decorator(cos)

cos(3.1415)

Do something additional...
-0.9999999957076562


### Decorators with Parameters

In [7]:
def greeting(expr):
    def greeting_decorator(func):
        def function_wrapper(x):
            print(expr)
            func(x)
        return function_wrapper
    return greeting_decorator

@greeting("Hello")
def foo(x):
    print(x)

foo("World")

@greeting("Goodbye")
def foo(x):
    print(x)

foo("World")

Hello
World
Goodbye
World


### Preserve Attributes of Decorated Function
In the example below, after decoration, the attributes of the original functions will be lost:

In [8]:
from math import cos

def our_decorator(func):
    def function_wrapper(*args, **kwargs):
        print("Do something additional...")
        res = func(*args, **kwargs)
        print(res)
    return function_wrapper

cos = our_decorator(cos)
print(cos.__name__)
print(cos.__doc__)
print(cos.__module__)

function_wrapper
None
__main__


We can save the original attributes of the function:

In [9]:
from math import cos

def our_decorator(func):
    def function_wrapper(*args, **kwargs):
        print("Do something additional...")
        res = func(*args, **kwargs)
        print(res)
    function_wrapper.__name__ = func.__name__
    function_wrapper.__doc__ = func.__doc__
    function_wrapper.__module__ = func.__module__
    return function_wrapper


cos = our_decorator(cos)
print(cos.__name__)
print(cos.__doc__)
print(cos.__module__)

cos
Return the cosine of x (measured in radians).
math


# Class Decorators
A **callable object** is an object which can be used and behaves like a function. It is possible to define classes in a way that the instances will be callable objects.<br>
When the instance is called "like a function", the `__call__` method will be called.

In [16]:
class our_decorator:
    def __init__(self, f):
        self.f = f
    def __call__(self, *args, **kwargs):
        print("Do something additional...")
        self.f(*args, **kwargs)

@our_decorator
def foo(x):
    print("Call foo with " + str(x))

foo("Hi")

Do something additional...
Call foo with Hi


# Use Cases
### Checking Arguments
The following factorial function will get into endless loop if zero, negative, or float argument is passed:

In [10]:
def factorial(n):
    if n==1:
        return 1
    else:
        return n*factorial(n-1)

factorial(3)

6

Let's use a decorator to ensure proper argument ispassed to the function:

In [11]:
def argument_test_natural_number(f):
    def helper(x):
        if type(x)==int and x>0:
            return f(x)
        else:
            raise Exception("Argument must be a positive integer.")
    return helper

@argument_test_natural_number
def factorial(n):
    if n==1:
        return 1
    else:
        return n*factorial(n-1)

factorial(-1)

Exception: Argument must be a positive integer.

### Counting Function Calls
In this example, the decorator counts the number of times a function has been called.

In [12]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0

    return helper

@call_counter
def succ(x):
    return x + 1

print(succ.calls)

for i in range(10):
    succ(i)
    
print(succ.calls)

0
10


### Memoization with Decorators
**Memoization** is a technique used in computing to spreed up programs.<br>This is accomplished by memorizing the calculation results of processed input, such as the results of function calls. If the same input or a function call with the same parameters is used, the previously stored results can be used again and unnecessary calculation are avoided.

In this example, we will improve the runtime of the `fib` function by adding a dictionary to memorize previously calculated values of the function.<br>
One way is to modify the `fib` function by explicitly adding a memoization component, which will impair the clarify and beauty of the original recursive implementation.<br>
So we will define and use a function `memoize` to decorate `fib`.

In [14]:
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

@memoize
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

We will use another decorator to exam the runtime:

In [15]:
from timeit import timeit

def runtime_decorator(func, *args, **kwargs):
    def helper():
        return func(*args, **kwargs)
    return timeit(helper, number=1)

n = 1000
runtime1 = runtime_decorator(fib,n)
runtime2 = runtime_decorator(fib,n)

print(runtime1)
print(runtime2)

0.001653399999995031
1.1999999998124622e-06
